# Slackformer: Basic transformer on WBSlack data
All logic taken from Stacey's notebook and packaged into functions

In [1]:
import weave
import slack_trainer

In [2]:
SLACK_DATA_DIR = "/Users/timothysweeney/Documents/jan_1_2022_slack_dump"
raw_data = slack_trainer.load_data(SLACK_DATA_DIR)

In [3]:
# config = {
#     "min_msg_count"  : 50, 
#     "train_frac"     : 0.8,
#     "use_baseline"   : False,
#     "vocab_size"     : 30000, 
#     "sequence_length": 200,
#     "embed_dim"      : 64,
#     "num_heads"      : 6,
#     "ff_dim"         : 64,
#     "optimizer"      : "adam",
#     "loss"           : "sparse_categorical_crossentropy",
#     "metrics"        : ["accuracy"],
#     "batch_size"     : 64, 
#     "epochs"         : 5
# }
config = {
    "min_msg_count"  : 100,
    "train_frac"     : 0.8,
    "use_baseline"   : False,
    "vocab_size"     : 50000,
    "sequence_length": 200,
    "dense_dim"      : 40,
    "embed_dim"      : 64,
    "num_heads"      : 6,
    "ff_dim"         : 64,
    "optimizer"      : "adam",
    "loss"           : "sparse_categorical_crossentropy",
    "metrics"        : ["accuracy"],
    "batch_size"     : 64,
    "epochs"         : 5
}

In [4]:
# Process the data
data = slack_trainer.process_data(users         = raw_data["users"], 
                                  messages      = raw_data["messages"], 
                                  min_msg_count = config["min_msg_count"], 
                                  train_frac    = config["train_frac"])

In [5]:
# Build & Compile the Model
if config["use_baseline"]:
    model = slack_trainer.make_baseline_model(data, 
                                               vocab_size      = config["vocab_size"], 
                                               sequence_length = config["sequence_length"])
else:
    model = slack_trainer.make_transformer_model(data, 
                                                   vocab_size      = config["vocab_size"], 
                                                   sequence_length = config["sequence_length"],
                                                   dense_dim       = config["dense_dim"],
                                                   embed_dim       = config["embed_dim"],
                                                   num_heads       = config["num_heads"],
                                                   ff_dim          = config["ff_dim"])
model.compile(optimizer = config["optimizer"], 
                       loss      = config["loss"], 
                       metrics   = config["metrics"])

2022-08-11 13:39:46.092872: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
# Train the model
results = slack_trainer.fit_model(model, data, 
                                  batch_size = config["batch_size"], 
                                  epochs     = config["epochs"])

Epoch 1/5
1265/1265 [==============================] - 609s 480ms/step - loss: 4.2720 - accuracy: 0.0749 - val_loss: 3.9383 - val_accuracy: 0.1194
Epoch 2/5
1265/1265 [==============================] - 614s 486ms/step - loss: 3.7321 - accuracy: 0.1639 - val_loss: 3.6379 - val_accuracy: 0.1729
Epoch 3/5
 328/1265 [======>.......................] - ETA: 6:58 - loss: 3.3156 - accuracy: 0.2387

In [ ]:
# Package model for inference
packaged_model = slack_trainer.package_model(model, data)

In [ ]:
# Save or Publish Model
# saved_model = weave.save(packaged_model)
saved_model = weave.publish(packaged_model)

In [ ]:
# Demonstrate calling
from weave.ecosystem import keras as weave_keras
weave_keras.call_string(saved_model, "i love weave")